In [1]:
# http://pytorch.org/
from os import path
#from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
#platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
#accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
#import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
#from sklearn.metrics import confusion_matrix
#from scikit-learn import multilabel_confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix, classification_report
import time
import os

In [3]:
from models_cnn import *

In [4]:
#torch.cuda.device_count()
device = "gpu" if torch.cuda.is_available() else "cpu"
print(device)

gpu


# training

In [5]:
#net = Net1()
net = Net2()
#net = Net3()
#net = NetCEI201()
net = VGG('VGG16')
print(net)
net.cuda()

data_batch_size = 4
start_time = time.time()
if __name__ == '__main__':
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset  = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    ## reduce training size -Í begin
    num_train = len(trainset)
    indices   = list(range(num_train))
    split     = int(np.floor(0.5 * num_train))
    np.random.seed(10)
    np.random.shuffle(indices)
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    ## reduce training size - end
    #trainloader = torch.utils.data.DataLoader(trainset,batch_size=data_batch_size, shuffle=True, num_workers=2)
    trainloader = torch.utils.data.DataLoader(trainset,batch_size=data_batch_size, sampler=train_sampler, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=data_batch_size, shuffle=False, num_workers=2)

    classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    #optimizer = optim.Adam(net.parameters(), lr=0.001)

    train = True
    #train = False
    if train:
        for epoch in range(10):
            running_loss = 0.0

            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                if device == "gpu":
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())  
                else: 
                    inputs, labels = Variable(inputs), Variable(labels)
                            
                optimizer.zero_grad()
                output = net(inputs)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()
                #running_loss += loss.data[0]
                running_loss += loss.data.item()

                if i % 2000 == 1999:
                    print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 2000))
                    running_loss = 0.0

        print('Finished Training')

        torch.save(net.state_dict(), './00_cifar10-Net-model')
print("--- %s seconds ---" % (time.time() - start_time))  

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

/home/alvin/notebook/pytorch_model_excercise/models_cnn.py:135: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.softmax(out)


[1,  2000] loss: 2.092
[1,  4000] loss: 1.911
[1,  6000] loss: 1.766
[2,  2000] loss: 1.666
[2,  4000] loss: 1.573
[2,  6000] loss: 1.491
[3,  2000] loss: 1.398
[3,  4000] loss: 1.354
[3,  6000] loss: 1.298
[4,  2000] loss: 1.193
[4,  4000] loss: 1.170
[4,  6000] loss: 1.154
[5,  2000] loss: 1.034
[5,  4000] loss: 1.033
[5,  6000] loss: 1.028
[6,  2000] loss: 0.933
[6,  4000] loss: 0.921
[6,  6000] loss: 0.931
[7,  2000] loss: 0.818
[7,  4000] loss: 0.846
[7,  6000] loss: 0.844
[8,  2000] loss: 0.740
[8,  4000] loss: 0.746
[8,  6000] loss: 0.761
[9,  2000] loss: 0.684
[9,  4000] loss: 0.678
[9,  6000] loss: 0.673
[10,  2000] loss: 0.573
[10,  4000] loss: 0.604
[10,  6000] loss: 0.617
Finished Training
--- 1456.6239936351776 seconds ---


# inference

In [6]:
net.load_state_dict(torch.load('./00_cifar10-Net-model'))

correct = 0
total = 0

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs, targets = data
    
    images = Variable(inputs.cuda()) if device == "gpu" else Variable(inputs)
    labels = Variable(labels.cuda()) if device == "gpu" else Variable(labels)

    output = net(images)

    _, predicted = torch.max(output.data, 1)
    c = predicted.eq(labels.data).squeeze()
    for j in range(4):
        label = int(targets[j])
        class_correct[label] += int(c[j])
        class_total[label] += 1
        total += 1
        correct += int(c[j])
    
print('correct = ' + str(correct))
print('total = ' + str(total))
print('Total test accuracy : %3.2f %%' %(100 * int(correct) / total))

for i in range(10):
    print('Accuracy of %5s : %3.2f %% on %d, %d' 
          %(classes[i], 100 * int(class_correct[i]) // int(class_total[i]), class_correct[i],class_total[i]))

correct = 7271
total = 10000
Total test accuracy : 72.71 %
Accuracy of plane : 79.00 % on 794, 1000
Accuracy of   car : 81.00 % on 812, 1000
Accuracy of  bird : 51.00 % on 517, 1000
Accuracy of   cat : 54.00 % on 542, 1000
Accuracy of  deer : 74.00 % on 749, 1000
Accuracy of   dog : 67.00 % on 675, 1000
Accuracy of  frog : 77.00 % on 777, 1000
Accuracy of horse : 81.00 % on 819, 1000
Accuracy of  ship : 77.00 % on 772, 1000
Accuracy of truck : 81.00 % on 814, 1000


# confusion matrix

In [7]:
net.load_state_dict(torch.load('./00_cifar10-Net-model'))

correct = 0
total = 0

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

cf_label = []
cf_pred = []


for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs, targets = data

    images = Variable(inputs.cuda())
    labels = Variable(labels.cuda())
    #images = Variable(inputs)
    #labels = Variable(labels)

    output = net(images)

    _, predicted = torch.max(output.data, 1)
    c = predicted.eq(labels.data).squeeze()
    for j in range(4):
        label = int(targets[j])
        class_correct[label] += int(c[j])
        class_total[label] += 1
        total += 1
        correct += int(c[j])
    
    cf_label += labels.tolist()                           # <---- here
    cf_pred += predicted.tolist()                         # <---- here
print('correct = ' + str(correct))
print('total = ' + str(total))
print('Total test accuracy : %3.2f %%' %(100 * int(correct) / total))

for i in range(10):
    print('Accuracy of %5s : %3.2f %% on %d, %d' 
          %(classes[i], 100 * int(class_correct[i]) // int(class_total[i]), class_correct[i],class_total[i]))
    
cf = confusion_matrix(cf_label, cf_pred)  # <---- here
print('----------------------------\nconfusion matrix')   # <---- here
print(cf)                                                 # <---- here

correct = 7277
total = 10000
Total test accuracy : 72.77 %
Accuracy of plane : 78.00 % on 786, 1000
Accuracy of   car : 82.00 % on 820, 1000
Accuracy of  bird : 52.00 % on 521, 1000
Accuracy of   cat : 54.00 % on 540, 1000
Accuracy of  deer : 74.00 % on 744, 1000
Accuracy of   dog : 67.00 % on 675, 1000
Accuracy of  frog : 77.00 % on 775, 1000
Accuracy of horse : 82.00 % on 821, 1000
Accuracy of  ship : 78.00 % on 781, 1000
Accuracy of truck : 81.00 % on 814, 1000
----------------------------
confusion matrix
[[786   2  73  35  25   3   4  15  35  22]
 [ 27 820   2  20   1   3  11   6  27  83]
 [ 60   2 521 126 100 105  54  26   2   4]
 [ 15   5  46 540  68 221  47  47   5   6]
 [ 18   0  38  58 744  33  31  78   0   0]
 [  6   0  24 168  46 675  14  65   2   0]
 [  4   1  28 129  35  16 775   3   7   2]
 [ 11   1  11  38  51  64   1 821   0   2]
 [125   9  13  27   3   7   9   9 781  17]
 [ 48  49   3  26   2   4   7  35  12 814]]


In [8]:
cf = confusion_matrix(cf_label, cf_pred)
cf

array([[786,   2,  73,  35,  25,   3,   4,  15,  35,  22],
       [ 27, 820,   2,  20,   1,   3,  11,   6,  27,  83],
       [ 60,   2, 521, 126, 100, 105,  54,  26,   2,   4],
       [ 15,   5,  46, 540,  68, 221,  47,  47,   5,   6],
       [ 18,   0,  38,  58, 744,  33,  31,  78,   0,   0],
       [  6,   0,  24, 168,  46, 675,  14,  65,   2,   0],
       [  4,   1,  28, 129,  35,  16, 775,   3,   7,   2],
       [ 11,   1,  11,  38,  51,  64,   1, 821,   0,   2],
       [125,   9,  13,  27,   3,   7,   9,   9, 781,  17],
       [ 48,  49,   3,  26,   2,   4,   7,  35,  12, 814]])

In [9]:
cf = multilabel_confusion_matrix(cf_label, cf_pred)
cf

array([[[8686,  314],
        [ 214,  786]],

       [[8931,   69],
        [ 180,  820]],

       [[8762,  238],
        [ 479,  521]],

       [[8373,  627],
        [ 460,  540]],

       [[8669,  331],
        [ 256,  744]],

       [[8544,  456],
        [ 325,  675]],

       [[8822,  178],
        [ 225,  775]],

       [[8716,  284],
        [ 179,  821]],

       [[8910,   90],
        [ 219,  781]],

       [[8864,  136],
        [ 186,  814]]])

In [10]:
classes = ['plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck']
cf_label_name = list(map(lambda x: classes[x], cf_label))
cf_pred_name =  list(map(lambda x: classes[x], cf_pred))
report = classification_report(cf_label_name, cf_pred_name, labels=classes)
print(report)

              precision    recall  f1-score   support

       plane       0.71      0.79      0.75      1000
         car       0.92      0.82      0.87      1000
        bird       0.69      0.52      0.59      1000
         cat       0.46      0.54      0.50      1000
        deer       0.69      0.74      0.72      1000
         dog       0.60      0.68      0.63      1000
        frog       0.81      0.78      0.79      1000
       horse       0.74      0.82      0.78      1000
        ship       0.90      0.78      0.83      1000
       truck       0.86      0.81      0.83      1000

    accuracy                           0.73     10000
   macro avg       0.74      0.73      0.73     10000
weighted avg       0.74      0.73      0.73     10000

